In [33]:
from langgraph.graph import StateGraph, START ,END
from typing import TypedDict , Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages 


In [34]:
class ChatState(TypedDict):
  message : Annotated[list[BaseMessage], add_messages]

In [35]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
llm.invoke("HI")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--e148933b-7db8-4045-b4dc-7c327585314f-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [36]:
def chat_node(state : ChatState):
  message  = state["message"]
  
  response = llm.invoke(message)
  
  return {'message' : [response]}
  
  

In [37]:
checkpointer = MemorySaver()

graph  = StateGraph(ChatState)

graph.add_node("chat_node" , chat_node)
graph.add_edge(START , "chat_node")
graph.add_edge("chat_node" , END)

chatbot = graph.compile(checkpointer)

In [ ]:
thread_id = 1

while True:
  user_message = input("type here : ")
  print("User : ", user_message )
  
  if user_message.strip().lower() in ["bye" , "end" ,"quit"]:
    break
  
  config = {"configurable" : {"thread_id": thread_id}}
  
  response = chatbot.invoke({"message" : [HumanMessage(content = user_message)]} , config = config)
  
  print("AI : " , response["message"][-1].content)

User :  I am Kunj 
AI :  It's nice to meet you, Kunj.  Is there anything I can help you with today?
User :  can u explain the meaning of my name '
AI :  The name Kunj has a few possible meanings and origins, depending on the language and cultural context.  The most common interpretations are:

* **Sanskrit:**  In Sanskrit, Kunj often refers to a **grove, a cluster of trees, or a bower.**  This evokes a sense of beauty, serenity, and natural abundance.

* **Hindi/Urdu:**  It shares the same meaning as in Sanskrit, carrying the imagery of a **pleasant, secluded place, often associated with tranquility and peace.**

* **Other interpretations:**  In some contexts, it might also be seen as a variation or diminutive of other names.

Therefore, the meaning of your name Kunj generally points towards **nature, peace, and a secluded, beautiful place.**  It's a name that suggests a calm and peaceful personality.


In [21]:
initial_state = { 'message' : "what is the greatest number in Maths " }

ans = chat_bot.invoke(initial_state)

In [23]:
ans["message"][-1].content

"There is no greatest number in mathematics.  No matter what number you can think of, you can always add 1 to it to get a larger number.  This applies to even incredibly large numbers like Graham's number or TREE(3)."